### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 去標點

In [1]:
import json

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2]
    # 移除所有標點符號
    case_content = case_content.replace(" ","")
    case_content = case_content.replace("，","")
    case_content = case_content.replace("。","")
    case_content = case_content.replace(")","")
    case_content = case_content.replace("(","")
    case_content = case_content.replace("、","")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
    ne_index_list = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list.append((ne_idx,len(ne)))
    
    '''
    if len(main_list) != len(ne_index_list):
        c += 1
        #print(case_content)
        print(main_list)
        for i,l in ne_index_list:
            print(case_content[i])
        print("------------")
    '''
    
    texts = []
    for i,w in enumerate(case_content):
        label = getLabelWithWordIndex(i,ne_index_list)
        texts.append((w,label))
        
    docs.append(texts)

print(t)
print(c)

13452
0


In [5]:
import nltk

In [6]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])


[('李', 'JJ', 'O'), ('某', 'NNP', 'O'), ('女', 'NNP', 'O'), ('6', 'CD', 'O'), ('7', 'CD', 'O'), ('岁', 'NN', 'O'), ('每', 'NNP', 'O'), ('于', 'NNP', 'O'), ('夏', 'NNP', 'O'), ('日', 'NNP', 'O'), ('洗', 'NNP', 'O'), ('澡', 'NNP', 'O'), ('后', 'NNP', 'O'), ('即', 'NNP', 'O'), ('寒', 'NNP', 'B-S'), ('热', 'NNP', 'I-S'), ('不', 'NNP', 'I-S'), ('适', 'NNP', 'E-S'), ('自', 'NNP', 'O'), ('感', 'NNP', 'O'), ('发', 'NNP', 'O'), ('热', 'NNP', 'O'), ('而', 'NNP', 'O'), ('又', 'NNP', 'O'), ('需', 'NNP', 'O'), ('披', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('裹', 'NNP', 'O'), ('护', 'NNP', 'O'), ('并', 'NNP', 'O'), ('于', 'NNP', 'O'), ('不', 'NNP', 'O'), ('知', 'NNP', 'O'), ('不', 'NNP', 'O'), ('觉', 'NNP', 'O'), ('中', 'NNP', 'O'), ('汗', 'NNP', 'B-S'), ('透', 'NNP', 'I-S'), ('衣', 'NNP', 'I-S'), ('服', 'NNP', 'E-S'), ('急', 'NNP', 'O'), ('又', 'NNP', 'O'), ('换', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('如', 'NNP', 'O'), ('此', 'NNP', 'O'), ('三', 'NNP', 'O'), ('四', 'NNP', 'O'), ('次', 'NNP', 'O'), ('方', 'NNP', 'O'), ('渐', 'NNP', 'O'), ('趋', 'NNP', 'O'),

In [16]:
print(len(data))

2107


In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30691
Seconds required: 2.737

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 195969.764161
Feature norm: 1.000000
Error norm: 86317.429117
Active features: 30557
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.952

***** Iteration #2 *****
Loss: 193816.718536
Feature norm: 1.016924
Error norm: 73987.264294
Active features: 29335
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.480

***** Iteration #3 *****
Loss: 184556.229206
Feature norm: 1.114768
Error norm: 55816.042885
Active features: 22433
Line search trials: 1
Line search step: 1.000000
Seconds required 

***** Iteration #43 *****
Loss: 41719.650379
Feature norm: 24.321960
Error norm: 68575.745211
Active features: 25812
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.482

***** Iteration #44 *****
Loss: 40691.460262
Feature norm: 24.994379
Error norm: 14718.049029
Active features: 25767
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.480

***** Iteration #45 *****
Loss: 40426.622834
Feature norm: 25.543018
Error norm: 28323.858823
Active features: 25754
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.961

***** Iteration #46 *****
Loss: 40194.884818
Feature norm: 25.518163
Error norm: 7161.567514
Active features: 25765
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.479

***** Iteration #47 *****
Loss: 40021.964728
Feature norm: 25.608452
Error norm: 9149.580722
Active features: 25747
Line search trials: 1
Line search step: 1.000

***** Iteration #89 *****
Loss: 24220.169766
Feature norm: 83.569018
Error norm: 2306.181373
Active features: 22362
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.482

***** Iteration #90 *****
Loss: 24142.034823
Feature norm: 84.374537
Error norm: 5238.426866
Active features: 22369
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.490

***** Iteration #91 *****
Loss: 24047.918166
Feature norm: 85.173144
Error norm: 4092.878324
Active features: 22162
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.482

***** Iteration #92 *****
Loss: 24001.420467
Feature norm: 86.422480
Error norm: 14840.655235
Active features: 22132
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.477

***** Iteration #93 *****
Loss: 23936.025300
Feature norm: 86.704689
Error norm: 1951.256757
Active features: 22272
Line search trials: 1
Line search step: 1.00000

***** Iteration #132 *****
Loss: 22898.817917
Feature norm: 108.552749
Error norm: 4072.208801
Active features: 21408
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.480

***** Iteration #133 *****
Loss: 22891.465292
Feature norm: 108.790578
Error norm: 5572.010382
Active features: 21405
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.478

***** Iteration #134 *****
Loss: 22878.889244
Feature norm: 109.128975
Error norm: 3291.610239
Active features: 21395
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.481

***** Iteration #135 *****
Loss: 22869.842296
Feature norm: 109.273724
Error norm: 1993.178901
Active features: 21391
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.482

***** Iteration #136 *****
Loss: 22859.474825
Feature norm: 109.493195
Error norm: 2371.033815
Active features: 21333
Line search trials: 1
Line search step

***** Iteration #172 *****
Loss: 22595.511999
Feature norm: 112.873937
Error norm: 2393.254269
Active features: 20461
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.482

***** Iteration #173 *****
Loss: 22589.471264
Feature norm: 112.883683
Error norm: 1792.101777
Active features: 20441
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.477

***** Iteration #174 *****
Loss: 22584.766046
Feature norm: 112.949741
Error norm: 2956.456572
Active features: 20420
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.478

***** Iteration #175 *****
Loss: 22579.848680
Feature norm: 112.956944
Error norm: 1467.273316
Active features: 20383
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.491

***** Iteration #176 *****
Loss: 22575.160255
Feature norm: 113.026849
Error norm: 2008.981127
Active features: 20322
Line search trials: 1
Line search step

In [10]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

初 (O)
诊 (O)
： (O)
2 (O)
0 (O)
0 (O)
5 (O)
年 (O)
9 (O)
月 (O)
1 (O)
3 (O)
日 (O)
倪 (O)
某 (O)
2 (O)
9 (O)
岁 (O)
原 (O)
有 (O)
慢 (O)
性 (O)
盆 (O)
腔 (O)
炎 (O)
子 (O)
宫 (O)
内 (O)
膜 (O)
异 (O)
位 (O)
症 (O)
子 (O)
宫 (O)
肌 (O)
瘤 (O)
病 (O)
史 (O)
因 (O)
要 (O)
求 (O)
生 (O)
育 (O)
就 (O)
诊 (O)
末 (O)
次 (O)
月 (O)
经 (O)
8 (O)
月 (O)
9 (O)
日 (O)
～ (O)
8 (O)
月 (O)
1 (O)
5 (O)
日 (O)
8 (O)
月 (O)
1 (O)
9 (O)
日 (O)
开 (O)
始 (O)
阴 (B-S)
道 (I-S)
滴 (I-S)
沥 (I-S)
渗 (I-S)
血 (E-S)
至 (O)
今 (O)
不 (O)
绝 (O)
咖 (O)
啡 (O)
色 (O)
时 (O)
或 (O)
夹 (O)
带 (O)
恶 (O)
心 (O)
舌 (O)
稍 (O)
淡 (O)
苔 (O)
薄 (O)
腻 (O)
脉 (O)
细 (O)
治 (O)
法 (O)
： (O)
温 (O)
经 (O)
止 (O)
血 (O)
方 (O)
剂 (O)
： (O)
柏 (O)
叶 (O)
汤 (O)
合 (O)
理 (O)
中 (O)
汤 (O)
桃 (O)
花 (O)
汤 (O)
加 (O)
减 (O)
侧 (O)
柏 (O)
叶 (O)
1 (O)
0 (O)
g (O)
艾 (O)
叶 (O)
炭 (O)
5 (O)
g (O)
党 (O)
参 (O)
1 (O)
2 (O)
g (O)
炒 (O)
白 (O)
术 (O)
1 (O)
2 (O)
g (O)
炙 (O)
甘 (O)
草 (O)
6 (O)
g (O)
炮 (O)
姜 (O)
5 (O)
g (O)
赤 (O)
石 (O)
脂 (O)
1 (O)
5 (O)
g (O)
阿 (O)
胶 (O)
（ (O)
烊 (O)
冲 (O)
） (O)
1 (O)
0 (O)
g (O)
仙 (O)
鹤 (O)
草 (O)
2 (O

In [11]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-S","I-S","E-S"]))

             precision    recall  f1-score   support

          O       0.97      1.00      0.98    287954
        B-S       0.46      0.13      0.20      2582
        I-S       0.44      0.13      0.20      3877
        E-S       0.48      0.13      0.21      2571

avg / total       0.96      0.97      0.96    296984

